# MICrONS NWB co-registration and visualization

This notebook uses the `dandi`-hosted [MICrONS functional data](https://dandiarchive.org/dandiset/000402/draft) and `bossdb`-hosted [MICrONS structural data](https://bossdb.org/microns/minnie) to examine and visualize the co-registered cells.


## Data Access

Here we use the DANDI client, CAVE client, and CloudVolume client to establish access to the functional data, annotation data, and structural data respectively. 

In [74]:
from dandi.dandiapi import DandiAPIClient
from caveclient import CAVEclient
from cloudvolume import CloudVolume

from fsspec.implementations.cached import CachingFileSystem
from fsspec import filesystem
from h5py import File
from pynwb import NWBHDF5IO

import pandas as pd

In [4]:
cave = CAVEclient("minnie65_phase3_v1")

### Sub-task

Find the best NWB file to use

In [5]:
cave.materialize.get_tables()

['synapses_pni_2',
 'nucleus_detection_v0',
 'allen_minnie_extra_types',
 'aibs_soma_nuc_metamodel_preds_v117',
 'bodor_pt_target_proofread',
 'layer5_it_proofreading',
 'allen_v1_column_thalamic',
 'baylor_gnn_cell_type_fine_model_v2',
 'apl_functional_coreg',
 'nucleus_alternative_points',
 'allen_column_mtypes_v2',
 'column_atype',
 'connectivity_groups_v507',
 'new_table_test',
 'proofreading_functional_coreg_v2',
 'proofreading_functional_coreg',
 'bodor_pt_cells',
 'functional_coreg',
 'proofreading_apl_multisoma',
 'aibs_metamodel_mtypes_v661_v2',
 'proofreading_status_public_release',
 'allen_column_l5it_types',
 'allen_v1_column_types_slanted',
 'baylor_e_i_model_v1',
 'allen_column_mtypes_v1',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'apl_functional_coreg_v2',
 'coregistration_manual',
 'aibs_soma_nuc_exc_mtype_preds_v117',
 'baylor_log_reg_cell_type_coarse_v1',
 'apl_functional_coreg_forward_v3',
 'coregistration_manua

In [6]:
coreg = cave.materialize.query_table("apl_functional_coreg_forward_v5")

In [37]:
groups = coreg.groupby(['session', 'scan_idx'])
print("(session, scan) n_matches  n_seg_ids")
for key, item in groups:
    print(key, item.shape[0], item['pt_root_id'].unique().shape[0])

(session, scan) n_matches  n_seg_ids
(4, 7) 3499 3441
(4, 9) 3537 3466
(4, 10) 4291 4206
(5, 3) 3564 3507
(5, 6) 3886 3831
(5, 7) 3707 3654
(6, 2) 3634 3605
(6, 4) 3692 3661
(6, 6) 3515 3472
(6, 7) 3614 3579
(7, 3) 3585 3547
(7, 4) 5441 5385
(7, 5) 3068 3038
(8, 5) 4301 3872
(8, 7) 3020 2673
(8, 9) 3204 2809
(9, 3) 3514 2848
(9, 4) 3494 2804
(9, 6) 2307 2305


Session 7 scan 4 has the highest amount of matches

In [59]:
coreg.head()


,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
0,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,67007,2023-04-06 15:05:50.491194+00:00,t,255686,9,6,1913,2,8.941486,4.206369,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
1,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,65810,2023-04-06 15:05:49.502144+00:00,t,255686,9,4,7548,6,9.380808,-5.165012,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
2,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,54982,2023-04-06 15:05:15.938520+00:00,t,255686,8,7,4778,4,3.763804,5.341737,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
3,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,46412,2023-04-06 15:04:43.197444+00:00,t,255686,7,5,1946,2,4.071072,3.306988,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
4,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,11823,2023-04-06 15:03:01.038836+00:00,t,255686,5,3,1994,2,5.419972,6.644126,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"


### Subtask 3: Explore NWB File

In [39]:
dandiset_id = "000402"
file_path = "sub-17797/sub-17797_ses-7-scan-4_behavior+image+ophys.nwb" # file size ~67GB

# Get the location of the file on DANDI
with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(file_path)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

In [43]:
microns_data

root pynwb.file.NWBFile at 0x5142908048
Fields:
  acquisition: {
    EyeTracking <class 'pynwb.behavior.EyeTracking'>,
    PupilTracking <class 'pynwb.behavior.PupilTracking'>,
    TwoPhotonSeries1 <class 'pynwb.ophys.TwoPhotonSeries'>,
    TwoPhotonSeries2 <class 'pynwb.ophys.TwoPhotonSeries'>,
    TwoPhotonSeries3 <class 'pynwb.ophys.TwoPhotonSeries'>,
    TwoPhotonSeries4 <class 'pynwb.ophys.TwoPhotonSeries'>,
    TwoPhotonSeries5 <class 'pynwb.ophys.TwoPhotonSeries'>,
    TwoPhotonSeries6 <class 'pynwb.ophys.TwoPhotonSeries'>,
    TwoPhotonSeries7 <class 'pynwb.ophys.TwoPhotonSeries'>,
    TwoPhotonSeries8 <class 'pynwb.ophys.TwoPhotonSeries'>,
    Video: stimulus_17797_7_4_v4 <class 'pynwb.image.ImageSeries'>,
    treadmill_velocity <class 'pynwb.base.TimeSeries'>
  }
  devices: {
    Microscope <class 'pynwb.device.Device'>
  }
  experiment_description: The light microscopic images were acquired from a cubic millimeter volume that spanned portions of primary visual cortex and three higher visual cortical areas. The volume was imaged in vivo by two-photon random access mesoscope (2P-RAM) from postnatal days P75 to P81 in a male mouse expressing a genetically encoded calcium indicator in excitatory cells, while the mouse viewed natural movies and parametric stimuli. The calcium imaging data includes the single-cell responses of an estimated 75,000 pyramidal cells imaged over a volume of approximately 1200 x 1100 x 500 μm3 (anteroposterior x mediolateral x radial depth). The center of the volume was placed at the junction of primary visual cortex (VISp) and three higher visual areas, lateromedial area (VISlm), rostrolateral area (VISrl) and anterolateral area (VISal). During imaging, the animal was head-restrained, and the stimulus was presented to the left visual field. Treadmill rotation (single axis) and video of the animal's left eye were captured throughout the scan, yielding the locomotion velocity, eye movements, and pupil diameter data included here. The functional data were co-registered with electron microscopy (EM) data. The structural identifiers of the matched cells are added as plane segmentation columns extracted from the CAVE database. To access the latest revision see the notebook that is linked to this dandiset. The structural ids might not be present for all plane segmentations.
  file_create_date: [datetime.datetime(2023, 2, 13, 11, 47, 25, 63376, tzinfo=tzutc())]
  identifier: 4032004d-8ebd-42ab-9da1-e463a602ff00
  imaging_planes: {
    ImagingPlane1 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane2 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane3 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane4 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane5 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane6 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane7 <class 'pynwb.ophys.ImagingPlane'>,
    ImagingPlane8 <class 'pynwb.ophys.ImagingPlane'>
  }
  intervals: {
    Clip <class 'pynwb.epoch.TimeIntervals'>,
    Monet2 <class 'pynwb.epoch.TimeIntervals'>,
    Trippy <class 'pynwb.epoch.TimeIntervals'>
  }
  processing: {
    ophys <class 'pynwb.base.ProcessingModule'>
  }
  related_publications: ['https://doi.org/10.1101/2021.07.28.454025']
  session_description: Contains calcium imaging recorded from multiple cortical visual areas and behavioral measurements while a mouse viewed natural movies and parametric stimuli. The structural ids are added as plane segmentation columns from the CAVE database on 2023-02-13. To access the latest revision see the notebook that is linked to the dandiset. The structural ids might not be present for all plane segmentations.
  session_id: 7-scan-4
  session_start_time: 2018-03-07 00:00:00+00:00
  subject: subject pynwb.file.Subject at 0x5142979024
Fields:
  age: P75D/P81D
  sex: M
  species: Mus musculus
  subject_id: 17797

  timestamps_reference_time: 2018-03-07 00:00:00+00:00

### Update co-registration to automated table

In [61]:
image_segmentation = microns_data.processing["ophys"].data_interfaces["ImageSegmentation"]
image_segmentation

In [64]:
def update_plane_segmentation(nwb_file, coreg_table):
    pass

In [72]:
ps4 = image_segmentation["PlaneSegmentation4"][:]

In [73]:
ps4

,image_mask,mask_type,cave_ids,pt_supervoxel_id,pt_root_id,pt_x_position,pt_y_position,pt_z_position
id,,,,,,,,
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",artifact,[nan],NaN,NaN,NaN,NaN,NaN
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",artifact,[nan],NaN,NaN,NaN,NaN,NaN
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",soma,[nan],NaN,NaN,NaN,NaN,NaN
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",soma,[nan],NaN,NaN,NaN,NaN,NaN
5,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",soma,[nan],NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1619,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",soma,[8595.0],1.064055e+17,8.646911e+17,303041.0,120166.0,16756.0
1620,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",soma,[5129.0],9.015143e+16,8.646911e+17,184608.0,128560.0,25525.0
1621,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",soma,[nan],NaN,NaN,NaN,NaN,NaN


In [75]:
scan_unit = pd.read_pickle("ScanUnit.pkl")

In [82]:
session_units = scan_unit[(scan_unit['session']==7) & (scan_unit['scan_idx']==4) & (scan_unit['field'] == 4)]


In [83]:
ps4['mask_id']=ps4.index

In [87]:
ps4_units = ps4.merge(session_units, on='mask_id', how='left')

In [94]:
ps4_units.head()

,image_mask,mask_type,cave_ids,pt_supervoxel_id,pt_root_id,pt_x_position,pt_y_position,pt_z_position,mask_id,session,scan_idx,unit_id,field,um_x,um_y,um_z,px_x,px_y,ms_delay
0,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",artifact,[nan],NaN,NaN,NaN,NaN,NaN,1,7,4,4680,4,-268,-788,200,3,3,58
1,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",artifact,[nan],NaN,NaN,NaN,NaN,NaN,2,7,4,4681,4,-189,-789,200,34,2,58
2,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",soma,[nan],NaN,NaN,NaN,NaN,NaN,3,7,4,4682,4,-159,-765,200,46,12,59
3,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",soma,[nan],NaN,NaN,NaN,NaN,NaN,4,7,4,4683,4,-109,-779,200,66,6,58
4,"[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...",soma,[nan],NaN,NaN,NaN,NaN,NaN,5,7,4,4684,4,-126,-772,200,59,9,58


In [88]:
coreg.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
0,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,67007,2023-04-06 15:05:50.491194+00:00,t,255686,9,6,1913,2,8.941486,4.206369,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
1,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,65810,2023-04-06 15:05:49.502144+00:00,t,255686,9,4,7548,6,9.380808,-5.165012,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
2,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,54982,2023-04-06 15:05:15.938520+00:00,t,255686,8,7,4778,4,3.763804,5.341737,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
3,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,46412,2023-04-06 15:04:43.197444+00:00,t,255686,7,5,1946,2,4.071072,3.306988,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"
4,255686,2020-09-28 22:40:42.632533+00:00,t,297.846047,88954888800920543,864691135568539372,11823,2023-04-06 15:03:01.038836+00:00,t,255686,5,3,1994,2,5.419972,6.644126,"[175760, 126480, 15504]","[nan, nan, nan]","[nan, nan, nan]"


In [89]:
coreg_units = coreg[(coreg['session']==7) & (coreg['scan_idx']==4) & (coreg['field'] == 4)]


In [90]:
coreg_units.head()

,id_ref,created_ref,valid_ref,volume,pt_supervoxel_id,pt_root_id,id,created,valid,target_id,session,scan_idx,unit_id,field,residual,score,pt_position,bb_start_position,bb_end_position
111,294576,2020-09-28 22:44:55.293087+00:00,t,296.565309,90644426661199216,864691135272867601,42488,2023-04-06 15:04:39.942627+00:00,t,294576,7,4,5439,4,5.631002,5.422910,"[188320, 131408, 21605]","[nan, nan, nan]","[nan, nan, nan]"
239,259052,2020-09-28 22:45:19.613777+00:00,t,375.514890,87970071188074044,864691135737785220,42575,2023-04-06 15:04:40.013451+00:00,t,259052,7,4,5546,4,8.242632,5.887168,"[168752, 128832, 24985]","[nan, nan, nan]","[nan, nan, nan]"
504,155142,2020-09-28 22:44:32.830760+00:00,t,261.975081,82060333090743602,864691136518542052,43231,2023-04-06 15:04:40.543685+00:00,t,155142,7,4,6302,4,5.051677,6.651311,"[125744, 137936, 20829]","[nan, nan, nan]","[nan, nan, nan]"
521,155050,2020-09-28 22:42:21.332498+00:00,t,241.485742,82552914165610309,864691135257139887,42882,2023-04-06 15:04:40.260941+00:00,t,155050,7,4,5903,4,13.207301,0.918060,"[129488, 138224, 19615]","[nan, nan, nan]","[nan, nan, nan]"
528,155307,2020-09-28 22:41:36.928466+00:00,t,294.244188,82552502117044761,864691136108732088,42631,2023-04-06 15:04:40.059032+00:00,t,155307,7,4,5615,4,3.807639,5.660515,"[129168, 135152, 21555]","[nan, nan, nan]","[nan, nan, nan]"


In [92]:
coreg_match = ps4_units.merge(coreg_units, on='unit_id')

In [93]:
coreg_match.iloc[50]

image_mask            [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,...
mask_type                                                          soma
cave_ids                                                       [6752.0]
pt_supervoxel_id_x                                 114145319742642592.0
pt_root_id_x                                       864691136378803712.0
pt_x_position                                                  359278.0
pt_y_position                                                  114596.0
pt_z_position                                                   24024.0
mask_id                                                              57
session_x                                                             7
scan_idx_x                                                            4
unit_id                                                            4736
field_x                                                               4
um_x                                                            